In [8]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm

# Загрузка данных
data = pd.read_csv("esg_data.csv") 
print(data.head())

         company              text     E      S     G   ESG   Год  \
0  газпром нефть  социальный отчёт  99.0  100.0  73.0  90.0  2003   
1  газпром нефть     2002-2003 год  99.0  100.0  73.0  90.0  2003   
2  газпром нефть      оао сибнефть  99.0  100.0  73.0  90.0  2003   
3  газпром нефть            2004 г  99.0  100.0  73.0  90.0  2003   
4  газпром нефть  социальный отчёт  99.0  100.0  73.0  90.0  2003   

                             subindustry  MOEX_flag  
0  Интегрированные нефтегазовые компании          1  
1  Интегрированные нефтегазовые компании          1  
2  Интегрированные нефтегазовые компании          1  
3  Интегрированные нефтегазовые компании          1  
4  Интегрированные нефтегазовые компании          1  


In [27]:
df = data[[20 <= len(text.split()) <= 1000 for text in data['text']]]

In [28]:
df.shape

(69794, 9)

In [ ]:
# Подготовка данных
texts = df["text"].tolist()  
ratings_E = df["E"].values 
ratings_S = df["S"].values
ratings_G = df["G"].values
ratings_ESG = df["ESG"].values

model_name = "DeepPavlov/rubert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [20]:
def get_embeddings(texts):
    embeddings = []
    for text in tqdm(texts, desc="Processing texts"):
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).numpy()
        embeddings.append(embedding)
    return np.vstack(embeddings)

# Получение эмбеддингов для всех текстов
text_embeddings = get_embeddings(texts)

Processing texts:   0%|          | 869/723018 [00:19<4:23:54, 45.61it/s]


KeyboardInterrupt: 

In [ ]:
np.save("text_embeddings.npy", text_embeddings)

In [31]:
text_embeddings = np.load("text_embeddings.npy")

In [17]:
text_embeddings